In [ ]:
import requests
import pandas as pd 
from bs4 import BeautifulSoup
import urllib.request
import re
from konlpy.tag import Okt

In [ ]:
pres=['https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=161967&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=highest&page=1',
'https://movie.naver.com/movie/bi/mi/pointWriteFormList.naver?code=161967&type=after&onlyActualPointYn=N&onlySpoilerPointYn=N&order=lowest&page=1']

In [ ]:
#데이터양 

review = []
rate =[]
target=[]

for pre in pres:
    for i in range(1,200):
        url=pre+str(i)
        res=requests.get(url)
        soup=BeautifulSoup(res.content,'html.parser')
        
        id_list=[]
        id_pre='_filtered_ment_'
        
        for i in range(10):
            id_list.append(id_pre+str(i))
        
        for id in id_list:
            review.append(soup.find('span',{'id':id}).get_text().strip())
            
        rate_list =[]
        rate_list =(soup.select('div.star_score > em'))
        
        for i in range(10):
            r = int(re.sub('<.+?>','',str(rate_list[i])))
            rate.append(r)
            if r>=8:
                target.append(1)
            elif r<=4:
                target.append(0)
            else:
                target.append(-1)
        

df=pd.DataFrame({'review':review,'rate':rate,'target':target})
df

In [ ]:
df.rate.unique()

In [ ]:
from sklearn.model_selection import train_test_split

train_x,test_x,train_y,test_y=train_test_split(review,target,test_size=0.2,random_state=0)

len(train_x),len(train_y)
len(test_x),len(test_y)

In [ ]:
#텍스트 데이터의 백터화
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
okt=Okt()

tfv = TfidfVectorizer(tokenizer=okt.morphs,ngram_range=(1,2),min_df=3,max_df=0.9)
tfv.fit(train_x)

tfv_train_x = tfv.transform(train_x)
tfv_train_x


In [ ]:
#로지스틱 회귀 모형으로 모델 적합 
from sklearn.linear_model import LogisticRegression
from sklearn.experimental import enable_halving_search_cv 
from sklearn.model_selection import GridSearchCV

clf = LogisticRegression(random_state=0)
params = {'C':[1,3,5,7,9]}
grid_cv = GridSearchCV(clf,param_grid=params,cv=4,scoring="accuracy",verbose=1)
grid_cv.fit(tfv_train_x,train_y)

In [ ]:
#테스트 데이터를 통한 예측 정확도 산출 
tfv_test_x=tfv.transform(test_x)
grid_cv.best_estimator_.score(tfv_test_x,test_y)

In [ ]:
#영화 리뷰에 대한 긍부정 분류 
my_review = tfv.transform([input()])
if(grid_cv.best_estimator_.predict(my_review)):
    print("긍정 리뷰")
else:
    print("부정 리뷰")

In [ ]:
import numpy as np 
import matplotlib.pyplot as plt
from collections import Counter
okt = Okt()

In [ ]:
word_list = []
word_list = df["review"]

In [ ]:
#형태소 분리 
sentences_tag = []

for sentence in word_list: 
    morph = okt.pos(sentence)
    sentences_tag.append(morph)

In [ ]:
sentences_tag

In [ ]:
#명사만 추출 
noun_list = [] 
for sentence in sentences_tag:
    for word, tag in sentence:
        if tag in ["Noun"]:
            noun_list.append(word)

In [ ]:
noun_list

In [ ]:
len(noun_list)
#글자 길이가 1인것은 제외 
noun_list = [n for n in noun_list if len(n)>1] 
noun_list[:80]

In [ ]:
#단어별 갯수 세기 
counts = Counter(noun_list)
tags = counts.most_common(40)
tags

In [ ]:
#불필요한 단어 삭제
tags.remove(('그냥',119))
tags.remove(('보고',199))
tags.remove(('영화',2075))
tags.remove(('하나',89))

In [ ]:
from urllib import request
from wordcloud import WordCloud, ImageColorGenerator
import matplotlib.pyplot as plt
from PIL import Image

In [ ]:
from google.colab import files 
uploaded = files.upload()
!ls

In [ ]:
#워드클라우드 마스크 이미지 설정 
mask = np.array(Image.open("person_a.png"))

plt.figure(figsize=(8,8))
plt.imshow(mask)
plt.axis('off')
plt.show 

from wordcloud import  ImageColorGenerator
image_colors = ImageColorGenerator(mask)

In [ ]:
#워드클라우드 만들기/폰트 설정
from wordcloud import WordCloud 

In [ ]:
font_path = '/usr/share/fonts/truetype/nanum/NanumMyeongjoBold.ttf'
wc =WordCloud(font_path=font_path, background_color='white', width=800, height=600, mask=mask)
cloud = wc.generate_from_frequencies(dict(tags))
plt.figure(figsize=(10,8))
plt.axis('off')
plt.imshow(cloud)